In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('/content/games.csv', encoding='utf-8')
df.drop('Name',
  axis='columns', inplace=True)
df.drop('AppID',
  axis='columns', inplace=True)
df.replace('', pd.NA, inplace=True)

# Function to clean text data (e.g., removing special characters)
"""
def clean_text(text):
    if isinstance(text, str):
        # Remove non-ASCII characters
        text = text.encode('ascii', 'ignore').decode('ascii')
        # Further text preprocessing can be added here (e.g., removing special characters, lowercasing)
        return text
    return text

# Apply the cleaning function to all text columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].apply(clean_text)
    """

# Replace empty strings with NaN again, after cleaning
df.replace('', pd.NA, inplace=True)

# Convert all object columns to strings to avoid mixed types
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str)

# Define the target column and feature columns
target_column = 'Price'  # Replace with the actual target column name (e.g., 'Genre')
feature_columns = [col for col in df.columns if col != target_column]

# Separate features and target
X = df[feature_columns]
y = df[target_column]

# Handle missing values (convert '<NA>' to NaN)
X = X.replace('<NA>', pd.NA)
y = y.replace('<NA>', pd.NA)

# Convert categorical columns to numerical using LabelEncoder
label_encoders = {}
for col in X.select_dtypes(include=['category', 'object']).columns:
    le = LabelEncoder()
    X[col] = X[col].astype(str)  # Ensure all values are strings
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Standardize the feature columns (useful for some ML models)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [2]:

from sklearn.preprocessing import OneHotEncoder

# Convert all values in 'genre' to strings
df['Genres'] = df['Genres'].astype(str)

# Split multiple genres into separate entries
df['Genres'] = df['Genres'].str.split(',')

# Expand the DataFrame so that each genre gets its own row
df_expanded = df.explode('Genres').reset_index(drop=True)

# Initialize the OneHotEncoder
genre_ohe = OneHotEncoder(sparse=False)

# Fit and transform the 'genre' column
genre_encoded = genre_ohe.fit_transform(df_expanded[['Genres']])

# Convert the encoded data into a DataFrame for better visualization
genre_encoded_df = pd.DataFrame(genre_encoded, columns=genre_ohe.get_feature_names_out(['Genres']))

# Concatenate the original DataFrame with the one-hot encoded DataFrame
df_encoded = pd.concat([df_expanded, genre_encoded_df], axis=1)

# Drop the original 'genre' column if desired
df_encoded.drop('Genres', axis=1, inplace=True)

# Group by the original index to aggregate one-hot encoded genres for games with multiple genres
df_encoded = df_encoded.groupby(df_encoded.index).max()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
df=df_encoded
del df_encoded

In [4]:
df.drop("Header image",axis=1,inplace=True)
df.drop("Website",axis=1,inplace=True)
df.drop("Reviews",axis=1,inplace=True)
df.drop("Tags",axis=1,inplace=True)
df.drop("Screenshots",axis=1,inplace=True)


In [5]:
df["Support url"].replace("nan",0,inplace=True)
df['Support email'].replace('nan',0,inplace=True)
df['Support url'] = df['Support url'].where(df['Support url'] == 0, 1)
df['Support email'] = df['Support email'].where(df['Support email'] == 0, 1)
df.drop("About the game",axis=1,inplace=True)
df.drop("Metacritic url",axis=1,inplace=True)
df.drop("Notes",axis=1,inplace=True)

In [6]:
target_column = 'Price'  # Replace with the actual target column name (e.g., 'Genre')
feature_columns = [col for col in df.columns if col != target_column]

# Separate features and target
X = df[feature_columns]
y = df[target_column]
label_encoders = {}
for col in X.select_dtypes(include=['category', 'object']).columns:
    le = LabelEncoder()
    X[col] = X[col].astype(str)  # Ensure all values are strings
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-6-999eb27c4c34>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(str)  # Ensure all values are strings
<ipython-input-6-999eb27c4c34>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))
<ipython-input-6-999eb27c4c34>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)
print("training set score: {:.2f}".format(regr.score(X_train,y_train)))
print("test set score: {:.2f}".format(regr.score(X_test,y_test)))

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
regr = GradientBoostingRegressor(max_depth=22,random_state=0)
regr.fit(X_train, y_train)
print("training set score: {:.2f}".format(regr.score(X_train,y_train)))
print("test set score: {:.2f}".format(regr.score(X_test,y_test)))

training set score: 1.00
test set score: 0.90


In [210]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=22,n_estimators=150,random_state=42)
regr.fit(X_train,y_train)
print("training set score: {:.2f}".format(regr.score(X_train,y_train)))
print("test set score: {:.2f}".format(regr.score(X_test,y_test)))

training set score: 0.90
test set score: 0.81


In [270]:
from sklearn.linear_model import ElasticNet
regr = ElasticNet(alpha=0.0001,random_state=0)
regr.fit(X_train, y_train)
print("training set score: {:.2f}".format(regr.score(X_train,y_train)))
print("test set score: {:.2f}".format(regr.score(X_test,y_test)))

training set score: 0.09
test set score: 0.07


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.136e+06, tolerance: 2.537e+03
  model = cd_fast.enet_coordinate_descent(


In [249]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

model = keras.Sequential([
    layers.Dense(16, activation='sigmoid', input_shape=[60]),
    layers.Dense(160, activation='sigmoid'),
    layers.Dense(1600, activation='sigmoid'),
    layers.Dense(1600, activation='sigmoid'),
    layers.Dense(160, activation='sigmoid'),
    layers.Dense(16, activation='sigmoid'),
    layers.Dense(1,activation="softmax", name="predictions")
])
model.compile(optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss="mae",
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],)
X_train=tf.convert_to_tensor(X_train, dtype=tf.float32)  # Convert int32 to float32
y_train=tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test=tf.convert_to_tensor(X_test, dtype=tf.float32)  # Convert int32 to float32
y_test=tf.convert_to_tensor(y_test, dtype=tf.float32)
m=model.fit(
    X_train,y_train,
   validation_data=(X_test, y_test),
    batch_size=1024,
    epochs=10)

Epoch 1/10
122/185 [==================>...........] - ETA: 24s - loss: 6.5290 - sparse_categorical_accuracy: 0.1912

KeyboardInterrupt: 

In [209]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=22)
model.fit(X_train,y_train)
print('training set score :{:.2f}'.format(model.score(X_train,y_train)))
print("test set score: {:.2f}".format(model.score(X_test,y_test)))

training set score :0.84
test set score: 0.73


In [187]:
# Train and evaluate the model
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
model.fit(X_train, y_train)

print('training set score :{:.2f}'.format(model.score(X_train,y_train)))
print("test set score: {:.2f}".format(model.score(X_test,y_test)))

training set score :0.67
test set score: 0.56


In [188]:
from sklearn.linear_model import Ridge
ridge=Ridge(alpha=0.01,max_iter=100000).fit(X_train,y_train)
print("trainig set score :{:.2f}".format(ridge.score(X_train,y_train)))
print("test set score: {:.2f}".format(ridge.score(X_test,y_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.13177e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


trainig set score :0.10
test set score: 0.08


In [189]:
from sklearn.linear_model import LinearRegression
RL=LinearRegression().fit(X_train,y_train)
print('training set score :{:.2f}'.format(RL.score(X_train,y_train)))
print("test set score: {:.2f}".format(RL.score(X_test,y_test)))

training set score :0.10
test set score: 0.08
